In [1]:
#!pip install -U datasets optuna

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [2]:
from datasets import load_dataset
from transformers import set_seed, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score
import optuna
import pandas as pd

In [3]:
seed = 42
set_seed(seed)

In [4]:
train = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="train")
val = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="dev")
test = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="test")

In [5]:
emotion_cols = ['anger', 'fear', 'joy', 'disgust', 'sadness', 'surprise']

In [6]:
def create_labels(examples):
    labels = []
    for i in range(len(examples['text'])):
        label = [float(examples[col][i]) for col in emotion_cols]
        labels.append(label)
    examples['labels'] = labels
    return examples

train = train.map(create_labels, batched=True)
val = val.map(create_labels, batched=True)
test = test.map(create_labels, batched=True)

In [7]:
model_name = "DeepPavlov/rubert-base-cased-conversational"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=False, max_length=512)

train_tokenized = train.map(tokenize_function, batched=True)
val_tokenized = val.map(tokenize_function, batched=True)

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [9]:
train_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
    y_pred = predictions > 0.5

    results = {}
    for average in ['micro', 'macro']:
        results[f'{average}_recall'] = recall_score(labels, y_pred, average=average, zero_division=0)
        results[f'{average}_precision'] = precision_score(labels, y_pred, average=average, zero_division=0)
        results[f'{average}_f1'] = f1_score(labels, y_pred, average=average, zero_division=0)

    return results

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 500)
    num_epochs = trial.suggest_int("num_train_epochs", 2, 7)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(emotion_cols),
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=f'./results/trial_{trial.number}',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        logging_steps=50,
        eval_strategy="epoch",
        metric_for_best_model="eval_macro_f1",
        logging_dir=f'./logs/trial_{trial.number}',
        save_strategy="no",
        report_to=None,
        seed=seed,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results["eval_macro_f1"]

In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("\nBest hyperparameters:")
print(study.best_params)
print(f"Best macro F1: {study.best_value:.4f}")

[I 2025-06-13 11:52:09,852] A new study created in memory with name: no-name-a8c9761e-cc31-4f90-bf99-51cc96d486cb
/home/sofism/miniconda3/envs/python3.11.7/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations us

Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.413000,0.332565,0.243523,0.921569,0.385246,0.210811,0.471703,0.251958
2,0.201900,0.183849,0.818653,0.840426,0.829396,0.826916,0.854287,0.839073
3,0.107500,0.132825,0.813472,0.923529,0.865014,0.825965,0.923163,0.868217
4,0.065100,0.121228,0.834197,0.914773,0.872629,0.845267,0.917648,0.876775


[I 2025-06-13 11:53:02,401] Trial 0 finished with value: 0.8767750219225855 and parameters: {'learning_rate': 2.9556907845227148e-05, 'batch_size': 16, 'weight_decay': 0.2925600475266136, 'warmup_steps': 345, 'num_train_epochs': 4}. Best is trial 0 with value: 0.8767750219225855.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.661400,0.421520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.313000,0.225734,0.673575,0.890411,0.766962,0.669614,0.897478,0.743425


[I 2025-06-13 11:53:24,051] Trial 1 finished with value: 0.7434251011949037 and parameters: {'learning_rate': 5.097294412875871e-05, 'batch_size': 32, 'weight_decay': 0.0654128843393699, 'warmup_steps': 330, 'num_train_epochs': 2}. Best is trial 0 with value: 0.8767750219225855.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.466700,0.410509,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.275300,0.236483,0.663212,0.870748,0.752941,0.656677,0.892372,0.724262
3,0.155000,0.156400,0.813472,0.928994,0.867403,0.821610,0.934255,0.869977
4,0.096400,0.142663,0.808290,0.917647,0.859504,0.812111,0.931788,0.864090
5,0.072700,0.142179,0.823834,0.898305,0.859459,0.834224,0.904339,0.862655
6,0.061300,0.131762,0.834197,0.889503,0.860963,0.845518,0.902160,0.869484


[I 2025-06-13 12:13:45,705] Trial 18 finished with value: 0.8694840809209832 and parameters: {'learning_rate': 1.5460476763315463e-05, 'batch_size': 16, 'weight_decay': 0.11877940018507384, 'warmup_steps': 408, 'num_train_epochs': 6}. Best is trial 15 with value: 0.8946073182742188.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.173800,0.153930,0.782383,0.909639,0.841226,0.795996,0.911195,0.845193
2,0.106200,0.147999,0.797927,0.885057,0.839237,0.811277,0.886970,0.840757
3,0.077100,0.139129,0.839378,0.880435,0.859416,0.853302,0.879569,0.863031
4,0.040300,0.116752,0.865285,0.912568,0.888298,0.869354,0.913639,0.889317
5,0.022200,0.140474,0.823834,0.893258,0.857143,0.831037,0.895888,0.860577
6,0.015100,0.136804,0.854922,0.911602,0.882353,0.865853,0.916320,0.889682
7,0.006300,0.139276,0.854922,0.896739,0.875332,0.864498,0.900224,0.880579


[I 2025-06-13 12:16:18,813] Trial 19 finished with value: 0.8805786587235577 and parameters: {'learning_rate': 4.4713771999601775e-05, 'batch_size': 8, 'weight_decay': 0.22245469980190463, 'warmup_steps': 83, 'num_train_epochs': 7}. Best is trial 15 with value: 0.8946073182742188.



Best hyperparameters:
{'learning_rate': 1.913996384380825e-05, 'batch_size': 16, 'weight_decay': 0.2573376860542623, 'warmup_steps': 286, 'num_train_epochs': 6}
Best macro F1: 0.8946


In [14]:
best_params = study.best_params
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(emotion_cols),
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir='./best_model',
    num_train_epochs=best_params["num_train_epochs"],
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    weight_decay=best_params["weight_decay"],
    warmup_steps=best_params["warmup_steps"],
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir='./logs/best_model',
    seed=seed,
)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.411700,0.343846,0.264249,1.000000,0.418033,0.215836,0.500000,0.298086
2,0.213600,0.185761,0.766839,0.896970,0.826816,0.780510,0.906276,0.836338
3,0.121600,0.139141,0.803109,0.933735,0.863510,0.813314,0.934391,0.866683
4,0.077800,0.127788,0.849741,0.896175,0.872340,0.856785,0.915462,0.882698
5,0.056200,0.137023,0.834197,0.894444,0.863271,0.849181,0.903866,0.870918
6,0.047500,0.127227,0.849741,0.916201,0.881720,0.864039,0.927268,0.891356


TrainOutput(global_step=1008, training_loss=0.1778020536497472, metrics={'train_runtime': 74.6981, 'train_samples_per_second': 215.186, 'train_steps_per_second': 13.494, 'total_flos': 307607912530812.0, 'train_loss': 0.1778020536497472, 'epoch': 6.0})

In [21]:
def find_best_threshold(model, val_dataset, thresholds=np.arange(0.1, 0.9, 0.05)):
    model.eval()
    predictions = trainer.predict(val_dataset)
    probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
    true_labels = predictions.label_ids
    best_threshold = 0.5
    best_f1 = 0
    print("Threshold optimization:")
    for threshold in thresholds:
        y_pred = probs > threshold
        f1_macro = f1_score(true_labels, y_pred, average='macro', zero_division=0)
        print(f"Threshold {threshold:.2f}: Macro F1 = {f1_macro:.4f}")
        if f1_macro > best_f1:
            best_f1 = f1_macro
            best_threshold = threshold
    print(f"\nBest threshold: {best_threshold:.2f} (Macro F1: {best_f1:.4f})")
    return best_threshold

In [22]:
best_threshold = find_best_threshold(model, val_tokenized)

Threshold optimization:
Threshold 0.10: Macro F1 = 0.8174
Threshold 0.15: Macro F1 = 0.8610
Threshold 0.20: Macro F1 = 0.8696
Threshold 0.25: Macro F1 = 0.8751
Threshold 0.30: Macro F1 = 0.8695
Threshold 0.35: Macro F1 = 0.8812
Threshold 0.40: Macro F1 = 0.8860
Threshold 0.45: Macro F1 = 0.8878
Threshold 0.50: Macro F1 = 0.8914
Threshold 0.55: Macro F1 = 0.8886
Threshold 0.60: Macro F1 = 0.8834
Threshold 0.65: Macro F1 = 0.8677
Threshold 0.70: Macro F1 = 0.8561
Threshold 0.75: Macro F1 = 0.8505
Threshold 0.80: Macro F1 = 0.8468
Threshold 0.85: Macro F1 = 0.8245

Best threshold: 0.50 (Macro F1: 0.8914)


In [23]:
test_tokenized = test.map(tokenize_function, batched=True)
test_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

test_predictions = trainer.predict(test_tokenized)
test_probs = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()
test_pred_labels = test_probs > best_threshold
true_test_labels = test_predictions.label_ids

In [24]:
for average in ['micro', 'macro']:
    recall = recall_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    precision = precision_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    f1 = f1_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

print(f"\nPer-class Results:")
class_recall = recall_score(true_test_labels, test_pred_labels, average=None, zero_division=0)
class_precision = precision_score(true_test_labels, test_pred_labels, average=None, zero_division=0)
class_f1 = f1_score(true_test_labels, test_pred_labels, average=None, zero_division=0)

for i, emotion in enumerate(emotion_cols):
    print(f'{emotion.upper()}: recall: {round(class_recall[i], 4)}, precision: {round(class_precision[i], 4)}, f1: {round(class_f1[i], 4)}')

print(f"\nClass distribution in test set:")
for i, emotion in enumerate(emotion_cols):
    true_count = int(true_test_labels[:, i].sum())
    pred_count = int(test_pred_labels[:, i].sum())
    total = len(true_test_labels)
    print(f'{emotion.upper()}: true: {true_count}/{total} ({true_count/total:.1%}), predicted: {pred_count}/{total} ({pred_count/total:.1%})')

MICRO recall: 0.851, precision: 0.8993, f1: 0.8745
MACRO recall: 0.8502, precision: 0.8983, f1: 0.8734

Per-class Results:
ANGER: recall: 0.8496, precision: 0.8972, f1: 0.8727
FEAR: recall: 0.8889, precision: 0.9697, f1: 0.9275
JOY: recall: 0.886, precision: 0.9607, f1: 0.9218
DISGUST: recall: 0.8279, precision: 0.8783, f1: 0.8523
SADNESS: recall: 0.7872, precision: 0.8222, f1: 0.8043
SURPRISE: recall: 0.8618, precision: 0.8618, f1: 0.8618

Class distribution in test set:
ANGER: true: 452/2000 (22.6%), predicted: 428/2000 (21.4%)
FEAR: true: 216/2000 (10.8%), predicted: 198/2000 (9.9%)
JOY: true: 386/2000 (19.3%), predicted: 356/2000 (17.8%)
DISGUST: true: 244/2000 (12.2%), predicted: 230/2000 (11.5%)
SADNESS: true: 282/2000 (14.1%), predicted: 270/2000 (13.5%)
SURPRISE: true: 246/2000 (12.3%), predicted: 246/2000 (12.3%)
